In [17]:
import pandas as pd
import re

df = pd.read_csv('results.csv', encoding='CP949')
df

,파일명,선종,상태,위치
0,1.jpg,범장망,표류,LAT N33.22.872 LON E124.5.074
1,2.jpg,범장망,이동,LAT N32.48.241 LON E124.38.553
2,3.jpg,범장망,조업,LAT N32.55.893 LON E124.33.731
3,4.jpg,타망,이동,LAT N35.40.891 LON E124.25.481
4,5.jpg,타망,이동,LAT N32.23.668 LON E125.33.053
5,6.jpg,타망,이동,LAT N32.59.352 LON E124.29.609
6,7.jpg,유망,조업,LAT N34.54.232 LON E124.42.394
7,8.jpg,유망,조업,LAT N34.41.829 LON E124.20.120
8,9.jpg,유망,표류,LAT N36.56.279 LON E124.39.427
9,10.jpg,안강망,조업,LAT N34.15.543 LON E125.41.175


In [20]:
vessel_mapping = {
    '타망': 1,   # 중국 어선
    '유망': 2,   # 중국 어선
    '등광조망': 3,  # 중국 어선
    '범장망': 4,  # 중국 어선
    '안강망': 5,   # 한국 어선
    '연승': 6,   # 한국 어선
    '통발': 7,   # 한국 어선
    '저인망': 8,  # 한국 어선
    '낚시어선': 9, # 한국 어선
    '채낚기': 10  # 한국 어선
}
chinese_vessels = ['타망', '유망', '등광조망', '범장망'] # numeric = 0
korean_vessels = ['안강망', '연승', '통발', '저인망', '낚시어선', '채낚기'] #numeric = 1
state_mapping = {'조업':1,'이동':2,'표류':3}
df['state'] = df['상태'].str.strip()

df['vessel_type'] = df['선종'].map(vessel_mapping)
df['nation_type'] = df['선종'].apply(lambda x: 1 if x in korean_vessels else 0)
df['state'] = df['state'].apply(lambda x: x if x in state_mapping else None)
df['state'] = df['state'].map(state_mapping)

def extract_lat_lon(location_str):
    lat_match = re.search(r'LAT N(\d+\.\d+\.\d+)', location_str)
    lon_match = re.search(r'LON E(\d+\.\d+\.\d+)', location_str)
    
    if lat_match and lon_match:
        # 위도, 경도에서 숫자만 추출
        lat = lat_match.group(1).replace('.', '')  # .을 제거
        lon = lon_match.group(1).replace('.', '')  # .을 제거
        return lat, lon
    return None, None

# 새로운 위도와 경도 컬럼 생성
df['Latitude'], df['Longitude'] = zip(*df['위치'].apply(extract_lat_lon))

df.to_csv('preprocessed_dataset.csv', index=False)
print(df)

       파일명   선종    상태                               위치  state  vessel_type  \
0    1.jpg  범장망    표류    LAT N33.22.872 LON E124.5.074      3            4   
1    2.jpg  범장망    이동   LAT N32.48.241 LON E124.38.553      2            4   
2    3.jpg  범장망    조업   LAT N32.55.893 LON E124.33.731      1            4   
3    4.jpg   타망    이동   LAT N35.40.891 LON E124.25.481      2            1   
4    5.jpg   타망    이동   LAT N32.23.668 LON E125.33.053      2            1   
5    6.jpg   타망    이동   LAT N32.59.352 LON E124.29.609      2            1   
6    7.jpg   유망    조업   LAT N34.54.232 LON E124.42.394      1            2   
7    8.jpg   유망    조업   LAT N34.41.829 LON E124.20.120      1            2   
8    9.jpg   유망    표류   LAT N36.56.279 LON E124.39.427      3            2   
9   10.jpg  안강망    조업   LAT N34.15.543 LON E125.41.175      1            5   
10  11.jpg   통발  \t이동   LAT N34.17.855 LON E124.15.509      2            7   
11  12.jpg   통발    이동   LAT N33.41.223 LON E124.12.877      2   